# US Destinations: Part I

Since re-confirming the fact that the vast, vast majority of Knoedler's sales occurred with US buyers, I thought it would be interesting to get a bit more granular with the destination data and have a look at where specifically Knoedler works ended up in the US. Probably no surprises here—I presume we'll largely see New York City followed by other large US cities. All the same, it may very well be illuminating to see these sales mapped out.

In the last notebook, I extracted country names from the buyer locations field that I cleaned up. However, this field also contains city and state information, as well. And using the GeoPy library that I discovered, I can use this City, State, Country location information to generate coordinates, which is what I'll use to visualize the data. In this notebook, then, I am going to focus on just generating this new dataset which will consist of latitutde and longitude values for each unique US buyer location. In the next notebook, I'll merge those coordinates with our Knoedler dataset.

In [1]:
# Standard imports
%matplotlib inline

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Import cleaned dataset
knoedler = pd.read_csv('../data/knoedler_clean.csv')

# Get just buyer location info
buy_locs = knoedler['buy_auth_loc_1_single'].dropna()

buy_locs

/var/folders/_h/vtjqyqq96hv8fw2d_ybkq1sc0000gn/T/ipykernel_94966/1017419062.py:10: DtypeWarning: Columns (5,6,7,43,44,47,52,53,54,56,57,58,70,75,76,77,79,80,81,84,92) have mixed types. Specify dtype option on import or set low_memory=False.
  knoedler = pd.read_csv('../data/knoedler_clean.csv')


0            New York, NY, USA
1        Philadelphia, PA, USA
2        Philadelphia, PA, USA
3        Philadelphia, PA, USA
4            New York, NY, USA
                 ...          
40245        New York, NY, USA
40246        New York, NY, USA
40247        New York, NY, USA
40248     Saint Louis, MO, USA
40249    Philadelphia, PA, USA
Name: buy_auth_loc_1_single, Length: 27198, dtype: object

First, let's filter the dataframe for US locations.

In [2]:
buy_locs_us = buy_locs[buy_locs.str.contains(r', USA$')]

# Display location count and percentage of total
(
    pd.DataFrame({
        "count":pd.Series(buy_locs_us.value_counts(), dtype='int'),
        "percentage":pd.Series(buy_locs_us.value_counts()/len(buy_locs_us) * 100, dtype='float')
    })
    .style
    .format({'count':'{:,}', 'percentage':'{:.2f}%'})
)

,count,percentage
"New York, NY, USA","10,803",47.36%
"Philadelphia, PA, USA","1,347",5.91%
"Boston, MA, USA","1,132",4.96%
"Chicago, IL, USA",859,3.77%
"Brooklyn, NY, USA",559,2.45%
"Pittsburgh, PA, USA",473,2.07%
"Saint Louis, MO, USA",453,1.99%
"Baltimore, MD, USA",446,1.96%
"Milwaukee, WI, USA",360,1.58%
"Providence, RI, USA",355,1.56%


Already I can see my hypothesis is correct: Over 10,000 artworks from Knoedler's inventory ended up in New York City over the course of the dealer's history, which accounts for nearly 50% of its US buyers. After that, it's small potatoes: 6% of US buyers were in Philly,  5% in Boston, 4% in Chicago, etc.

To leverage the GeoPy library, all I need is a list of unique locations. Because I want to limit the number of requests I make, however, I am going to limit my GeoPy queries to cities that received at least five artworks from Knoedler. So, I'll prepare a new DataFrame with these cities along with columns for their matching coordinates when we get them.

In [3]:
buy_locs_us = buy_locs_us.value_counts()[buy_locs_us.value_counts() > 4]
buy_locs_us

New York, NY, USA          10803
Philadelphia, PA, USA       1347
Boston, MA, USA             1132
Chicago, IL, USA             859
Brooklyn, NY, USA            559
                           ...  
New Canaan, CT, USA            5
Duluth, MN, USA                5
Pomfret, CT, USA               5
Greensburg, PA, USA            5
Kennett Square, PA, USA        5
Name: buy_auth_loc_1_single, Length: 208, dtype: int64

Now, I'll prepare a new DataFrame that will include the location information as listed above (<code>city</code>), the location information as retrieved by GeoPy (<code>location</code>), and the coordinates information, also retrived by GeoPy (<code>coords</code>).

In [4]:
buy_locs_us = pd.DataFrame({
    'city': buy_locs_us.index,
    'location':'', 
    'coords':''})

buy_locs_us

,city,location,coords
0,"New York, NY, USA",,
1,"Philadelphia, PA, USA",,
2,"Boston, MA, USA",,
3,"Chicago, IL, USA",,
4,"Brooklyn, NY, USA",,
...,...,...,...
203,"New Canaan, CT, USA",,
204,"Duluth, MN, USA",,
205,"Pomfret, CT, USA",,
206,"Greensburg, PA, USA",,


Time to make our GeoPy queries. I'm concerned that the geolocator I specify, in this case Nominatim, will get cranky if I make all these requests at once, so I'll use some of GeoPy's throttling options to space out my requests and handle errors.

In [5]:
# Import GeoPy and initialize geolocator
from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="buyer-loc-points")

# Import GeoPy throttler and initialize
from geopy.extra.rate_limiter import RateLimiter
geocode = RateLimiter(geolocator.geocode, min_delay_seconds=1)

# Import progress bar and initialize
from tqdm import tqdm
tqdm.pandas()

# Get locations
buy_locs_us['location'] = buy_locs_us['city'].progress_apply(geocode)

# Get lat and lon points
buy_locs_us['coords'] = buy_locs_us['location'].progress_apply(lambda loc: tuple(loc.point) if loc else None)

buy_locs_us

100%|█████████████████████████████████████████████████████████| 208/208 [00:00<00:00, 203312.80it/s]


,city,location,coords
0,"New York, NY, USA","(City of New York, New York, United States, (4...","(40.7127281, -74.0060152, 0.0)"
1,"Philadelphia, PA, USA","(Philadelphia, Philadelphia County, Pennsylvan...","(39.9527237, -75.1635262, 0.0)"
2,"Boston, MA, USA","(Boston, Suffolk County, Massachusetts, United...","(42.3554334, -71.060511, 0.0)"
3,"Chicago, IL, USA","(Chicago, Cook County, Illinois, United States...","(41.8755616, -87.6244212, 0.0)"
4,"Brooklyn, NY, USA","(Brooklyn, Kings County, City of New York, New...","(40.6526006, -73.9497211, 0.0)"
...,...,...,...
203,"New Canaan, CT, USA","(New Canaan, Fairfield County, Connecticut, 06...","(41.146763, -73.4948446, 0.0)"
204,"Duluth, MN, USA","(Duluth, Saint Louis County, Minnesota, United...","(46.7729322, -92.1251218, 0.0)"
205,"Pomfret, CT, USA","(Pomfret, Windham County, Connecticut, United ...","(41.8975977, -71.9625736, 0.0)"
206,"Greensburg, PA, USA","(Greensburg, Westmoreland County, Pennsylvania...","(40.3014581, -79.5389289, 0.0)"


Let's check for any null values resulting from errors.

In [6]:
buy_locs_us[buy_locs_us.location.isnull()]

,city,location,coords
118,"Wesbury, NY, USA",None,None
129,"Sandspoint, NY, USA",None,None


Looks like we have some typos here: Westbury, not Wesbury; and Sands Point, not Sandspoint. I'll manually add the correct coordinates, but I'll leave the typos intact, since I'll be joining this coordinates DataFrame with the Knoedler one over the <code>city</code> column. Plus, the typos won't matter for our mapping purposes anyway.

In [9]:
buy_locs_us.iloc[118]['coords'] = (40.758889, -73.588056, 0.0)
buy_locs_us.iloc[129]['coords'] = (40.848611, -73.711667, 0.0)

buy_locs_us[buy_locs_us.coords.isnull()]

,city,location,coords


At this point, we're all finished. So that I don't have to re-perform these GeoPy queries later, I'll export this DataFrame to csv for use later.

In [10]:
buy_locs_us.to_csv('../data/knoedler_buyer_locs_us.csv')